This is a notebook to generate all the prompts and seeds in the prompts google sheet

In [2]:
!pip install diffusers
!pip install transformers scipy ftfy accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.4/716.4 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.1 MB/s eta 0:00:00


In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

connect_google_drive = True #@param {type:"boolean"}

if connect_google_drive:
    from google.colab import drive

    drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [4]:


import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4",
                                               torch_dtype=torch.float16,
                                               safety_checker=None
                                               )  


pipe = pipe.to("cuda")



Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


# Read In Google Sheet

In [7]:
import os 
import pandas as pd

code_folder = r'/content/gdrive/MyDrive/AI Music Visuals Share/New Codes'

prompt_sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1YhC-iejPPC0bV4OJlxqZIQJj3FJqM-6xYauY7TB8EQA').worksheet('prompts')

sheets = prompt_sheet.get_all_values()
df_prompt = pd.DataFrame(sheets[1:], columns=sheets[0]).set_index('name')

df_prompt

,prompt,seeds,guidance_scale
name,,,
succulent,"a giant succulent in the desert in the sand, t...",5842234013956479,5
reflected galaxy,underground lake with a galaxy reflected in th...,5883559517894420,50
red cave1,"descending deep into a cave, red lava on the w...",4866412378884770 4940561922459094,7.5
alien dunes,rolling dunes on an alien planet at night vivi...,5453396741977211 6583682090290432 648251554957...,7.5
cornu,"standing wave pattern, rainbow, wishing well",5842234013956479,50
alien mountain,"inspired by David A. Hardy, digital landscape ...",4092123727786949 7285707432238947,7.5
overgrown fantasy,a foggy view of a bridge in a fantasy overgrow...,5618257995059877 4638125635335835 823596687782...,7.5
falling in cave,"descending deep into a cave, red lava on the w...",5451964648290255,7.5
vine cave,a foggy view in a fantasy overgrown vines worl...,7078380737840840,7.5


# Iterate through prompts and seeds, outputting an image for both

In [ ]:
device = "cuda"
generator = torch.Generator(device=device)
width = 512
height = 512

output_folder = os.path.join(code_folder, 'output_prompt_seed')

if not os.path.exists(output_folder): os.makedirs(output_folder)

for name, row in df_prompt.iterrows():
  seeds = row['seeds'].split(' ')
  prompt = row['prompt']
  guidance_scale = float(row['guidance_scale'])

  for seed in seeds:
    output_fn = "{}_{}.png".format(name, seed)

    generator.manual_seed(int(seed))

    latent = torch.randn(
        (1, pipe.unet.in_channels, height // 8, width // 8),
        generator = generator,
        device = device
    )

    with torch.autocast(device):
      images = pipe(
          prompt,
          guidance_scale=guidance_scale,
          latents = latent,
      )

    output_image = images.images[0]

    output_image.save(os.path.join(output_folder, output_fn))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]